In [16]:
import pandas as pd
import numpy as np
from datetime import datetime
from dateutil import tz
import requests
import json
import matplotlib.pyplot as plt
import math
import scipy.stats as st

from dateutil import parser
import matplotlib.dates as mdates

In [2]:
def create_tuple(response):
    response_data = response.json()
    response_datum = response_data[0]
    response_target = response_datum['target']
    response_datapoints = response_datum['datapoints']
    tuple_array = [tuple(x) for x in response_datapoints]
    npa = np.array(tuple_array, dtype=[
        ('value', np.double), ('ts', 'datetime64[ms]')])
    return npa

In [3]:
"""
Returns an array of tuples where tuple[0] is the metric, and tuple[1] is the time 
of measurement

Target: a WebCTRL point name in the form of a string 
Example: "#biotech/biotech_4th_floor/fourth_floor_fume_hood_lab_spaces/lab_433_control/zone/zone_temp/trend_log"

Server: a string representing the WebCTRL server our point exists in
Example: "biotech_main"

Start: the start time we wish to be querying data about (in string format)
Example: "2021-12-31T11:00Z"

End: The end time we wish to be querying data about
Example: "2022-12-31T11:00Z"
"""
def fume_query(target,server, start,end):
  url = "https://ypsu0n34jc.execute-api.us-east-1.amazonaws.com/dev/query"
  data = {
      "range": {
        "from": start,
        "to": end,
      },
      "targets": [
        {
          "payload": {
            "schema": server,
          },
          "target": target
        }
      ],

    }
  request = requests.post(url, json=data)
  print(request)
  # print(request.json())
  return create_tuple(request)

# fume_query(target="#biotech/biotech_4th_floor/fourth_floor_fume_hood_lab_spaces/lab_433_control/hood_sash", server="biotech_main", start="2021-12-25T00:00:00.000", end="2021-12-25T00:00:00.000")
# fume_query(target="#biotech_weather/temperature_tn", server="biotech_main", start = str(datetime(2021, 11, 17, 1)), end = str(datetime(2021, 11, 17, 2)))

In [4]:
def query_to_list(point, server, start, end):
    master = fume_query(point, server, start, end)

    list = pd.Series(data=[i[0] for i in master], index=[i[1] for i in master])
    print("\n", point, "\n", list)

    list = list[~list.index.duplicated()]
    print("\n", point, " new\n", list)

    return list

In [23]:
# Arguments: Sash Point, Occ Point, Server Name, Start Time, End Time
# Returns: Total time that hood sash was open, aggregated by hour

def total_time_sash_open(sash_point, occ_point, server, start, end, is_occupied):
  sash_list = query_to_list(sash_point, server, start, end)
  occ_list = query_to_list(occ_point, server, start, end)

  df = pd.concat([sash_list, occ_list], axis=1)
  df.columns = ["sash", "occ"]
  display(df)

  time_interval = df.index[1].minute - df.index[0].minute

  # Figure out closed sash position
  # display(df["sash"].value_counts())

  # from running the above on a large time difference, 1.2 inches is the most common smallest value
  df["time_open_mins"] = np.where((df["sash"] > 1.2), time_interval, 0)

  df = df.dropna()
  df.index = df.index.map(lambda x: x.to_pydatetime().replace(tzinfo=tz.tzutc()).astimezone(tz.tzlocal()))
  display(df)

  df = df[df['occ']==int(is_occupied)]

  display(df)

  df = df.groupby(pd.Grouper(freq='60Min', label='right')).sum()

  return df["time_open_mins"]

sash_data_occ = total_time_sash_open(sash_point="#biotech/biotech_4th_floor/fourth_floor_fume_hood_lab_spaces/lab_433_control/hood_sash",
                                occ_point="#biotech/biotech_4th_floor/fourth_floor_fume_hood_lab_spaces/lab_433_control/occ_trend",
                                server="biotech_main",
                                start = str(datetime(2021, 11, 17, 1)),
                                end = str(datetime(2021, 11, 17, 2)),
                                is_occupied=True)

sash_data_unocc = total_time_sash_open(sash_point="#biotech/biotech_4th_floor/fourth_floor_fume_hood_lab_spaces/lab_433_control/hood_sash",
                                occ_point="#biotech/biotech_4th_floor/fourth_floor_fume_hood_lab_spaces/lab_433_control/occ_trend",
                                server="biotech_main",
                                start = str(datetime(2021, 11, 17, 1)),
                                end = str(datetime(2021, 11, 17, 2)),
                                is_occupied=False)

print(sash_data_occ)
print(sash_data_unocc)

final_df = pd.DataFrame(data={"occ": sash_data_occ, "unocc" : sash_data_unocc})
final_df = final_df.fillna(0)
final_df

<Response [200]>

 #biotech/biotech_4th_floor/fourth_floor_fume_hood_lab_spaces/lab_433_control/hood_sash 
 2021-11-17 05:00:00    3.3
2021-11-17 05:05:00    3.3
2021-11-17 05:10:00    3.3
2021-11-17 05:15:00    3.3
2021-11-17 05:20:00    3.3
                      ... 
2021-11-18 04:35:00    1.8
2021-11-18 04:40:00    1.8
2021-11-18 04:45:00    1.8
2021-11-18 04:50:00    1.8
2021-11-18 04:55:00    1.8
Length: 288, dtype: float64

 #biotech/biotech_4th_floor/fourth_floor_fume_hood_lab_spaces/lab_433_control/hood_sash  new
 2021-11-17 05:00:00    3.3
2021-11-17 05:05:00    3.3
2021-11-17 05:10:00    3.3
2021-11-17 05:15:00    3.3
2021-11-17 05:20:00    3.3
                      ... 
2021-11-18 04:35:00    1.8
2021-11-18 04:40:00    1.8
2021-11-18 04:45:00    1.8
2021-11-18 04:50:00    1.8
2021-11-18 04:55:00    1.8
Length: 288, dtype: float64
<Response [200]>

 #biotech/biotech_4th_floor/fourth_floor_fume_hood_lab_spaces/lab_433_control/occ_trend 
 2021-11-17 05:00:00    1.0
2021-11-17 0

,sash,occ
2021-11-17 05:00:00,3.3,1.0
2021-11-17 05:05:00,3.3,1.0
2021-11-17 05:10:00,3.3,1.0
2021-11-17 05:15:00,3.3,1.0
2021-11-17 05:20:00,3.3,1.0
...,...,...
2021-11-18 04:35:00,1.8,0.0
2021-11-18 04:40:00,1.8,0.0
2021-11-18 04:45:00,1.8,0.0
2021-11-18 04:50:00,1.8,0.0


,sash,occ,time_open_mins
2021-11-17 00:00:00-05:00,3.3,1.0,5
2021-11-17 00:05:00-05:00,3.3,1.0,5
2021-11-17 00:10:00-05:00,3.3,1.0,5
2021-11-17 00:15:00-05:00,3.3,1.0,5
2021-11-17 00:20:00-05:00,3.3,1.0,5
...,...,...,...
2021-11-17 23:35:00-05:00,1.8,0.0,5
2021-11-17 23:40:00-05:00,1.8,0.0,5
2021-11-17 23:45:00-05:00,1.8,0.0,5
2021-11-17 23:50:00-05:00,1.8,0.0,5


,sash,occ,time_open_mins
2021-11-17 00:00:00-05:00,3.3,1.0,5
2021-11-17 00:05:00-05:00,3.3,1.0,5
2021-11-17 00:10:00-05:00,3.3,1.0,5
2021-11-17 00:15:00-05:00,3.3,1.0,5
2021-11-17 00:20:00-05:00,3.3,1.0,5
...,...,...,...
2021-11-17 23:10:00-05:00,1.8,1.0,5
2021-11-17 23:15:00-05:00,1.8,1.0,5
2021-11-17 23:20:00-05:00,1.8,1.0,5
2021-11-17 23:25:00-05:00,1.8,1.0,5


<Response [200]>

 #biotech/biotech_4th_floor/fourth_floor_fume_hood_lab_spaces/lab_433_control/hood_sash 
 2021-11-17 05:00:00    3.3
2021-11-17 05:05:00    3.3
2021-11-17 05:10:00    3.3
2021-11-17 05:15:00    3.3
2021-11-17 05:20:00    3.3
                      ... 
2021-11-18 04:35:00    1.8
2021-11-18 04:40:00    1.8
2021-11-18 04:45:00    1.8
2021-11-18 04:50:00    1.8
2021-11-18 04:55:00    1.8
Length: 288, dtype: float64

 #biotech/biotech_4th_floor/fourth_floor_fume_hood_lab_spaces/lab_433_control/hood_sash  new
 2021-11-17 05:00:00    3.3
2021-11-17 05:05:00    3.3
2021-11-17 05:10:00    3.3
2021-11-17 05:15:00    3.3
2021-11-17 05:20:00    3.3
                      ... 
2021-11-18 04:35:00    1.8
2021-11-18 04:40:00    1.8
2021-11-18 04:45:00    1.8
2021-11-18 04:50:00    1.8
2021-11-18 04:55:00    1.8
Length: 288, dtype: float64
<Response [200]>

 #biotech/biotech_4th_floor/fourth_floor_fume_hood_lab_spaces/lab_433_control/occ_trend 
 2021-11-17 05:00:00    1.0
2021-11-17 0

,sash,occ
2021-11-17 05:00:00,3.3,1.0
2021-11-17 05:05:00,3.3,1.0
2021-11-17 05:10:00,3.3,1.0
2021-11-17 05:15:00,3.3,1.0
2021-11-17 05:20:00,3.3,1.0
...,...,...
2021-11-18 04:35:00,1.8,0.0
2021-11-18 04:40:00,1.8,0.0
2021-11-18 04:45:00,1.8,0.0
2021-11-18 04:50:00,1.8,0.0


,sash,occ,time_open_mins
2021-11-17 00:00:00-05:00,3.3,1.0,5
2021-11-17 00:05:00-05:00,3.3,1.0,5
2021-11-17 00:10:00-05:00,3.3,1.0,5
2021-11-17 00:15:00-05:00,3.3,1.0,5
2021-11-17 00:20:00-05:00,3.3,1.0,5
...,...,...,...
2021-11-17 23:35:00-05:00,1.8,0.0,5
2021-11-17 23:40:00-05:00,1.8,0.0,5
2021-11-17 23:45:00-05:00,1.8,0.0,5
2021-11-17 23:50:00-05:00,1.8,0.0,5


,sash,occ,time_open_mins
2021-11-17 01:50:00-05:00,3.3,0.0,5
2021-11-17 01:55:00-05:00,3.3,0.0,5
2021-11-17 02:00:00-05:00,3.3,0.0,5
2021-11-17 02:05:00-05:00,3.3,0.0,5
2021-11-17 02:10:00-05:00,3.3,0.0,5
...,...,...,...
2021-11-17 23:35:00-05:00,1.8,0.0,5
2021-11-17 23:40:00-05:00,1.8,0.0,5
2021-11-17 23:45:00-05:00,1.8,0.0,5
2021-11-17 23:50:00-05:00,1.8,0.0,5


2021-11-17 01:00:00-05:00    60
2021-11-17 02:00:00-05:00    50
2021-11-17 03:00:00-05:00    10
2021-11-17 04:00:00-05:00    60
2021-11-17 05:00:00-05:00    30
2021-11-17 06:00:00-05:00    20
2021-11-17 07:00:00-05:00    45
2021-11-17 08:00:00-05:00    20
2021-11-17 09:00:00-05:00    50
2021-11-17 10:00:00-05:00    60
2021-11-17 11:00:00-05:00    60
2021-11-17 12:00:00-05:00    60
2021-11-17 13:00:00-05:00    60
2021-11-17 14:00:00-05:00    60
2021-11-17 15:00:00-05:00    60
2021-11-17 16:00:00-05:00    60
2021-11-17 17:00:00-05:00    60
2021-11-17 18:00:00-05:00    60
2021-11-17 19:00:00-05:00    60
2021-11-17 20:00:00-05:00    60
2021-11-17 21:00:00-05:00    60
2021-11-17 22:00:00-05:00     5
2021-11-17 23:00:00-05:00    35
2021-11-18 00:00:00-05:00    30
Freq: 60T, Name: time_open_mins, dtype: int64
2021-11-17 02:00:00-05:00    10
2021-11-17 03:00:00-05:00    50
2021-11-17 04:00:00-05:00     0
2021-11-17 05:00:00-05:00    30
2021-11-17 06:00:00-05:00    40
2021-11-17 07:00:00-05:00 

,occ,unocc
2021-11-17 01:00:00-05:00,60,0.0
2021-11-17 02:00:00-05:00,50,10.0
2021-11-17 03:00:00-05:00,10,50.0
2021-11-17 04:00:00-05:00,60,0.0
2021-11-17 05:00:00-05:00,30,30.0
2021-11-17 06:00:00-05:00,20,40.0
2021-11-17 07:00:00-05:00,45,15.0
2021-11-17 08:00:00-05:00,20,40.0
2021-11-17 09:00:00-05:00,50,10.0
2021-11-17 10:00:00-05:00,60,0.0


In [22]:
# Arguments: CFM Point, Sash Point, Occ Point, Internal Temp Point, External Temp Point, Server Name, Start Time, End Time
# Returns: Total energy from hume hood, aggregated by hour.  Can be adapted later on to only show energy when unoccupied

# A switch in the energy calculations that allows us to avoid issues between times of years where it is hotter/colder outside.
def coldorhot(cfm, external, internal, time_interval):
    if external<=internal:
        #sensible heating equation
        return 1.08 * cfm * (internal - external) / (60 / time_interval)
    if external>internal:
        #enthalpy of air
        return 0.24 * cfm /13.333 * 60 * (external - internal) / (60 / time_interval)

def total_energy(cfm_point, sash_point, occ_point, internal_temp_point, external_temp_point, server, start, end, is_occupied):
  #external_temp_master = outside_temp(start,end)
  cfm_list = query_to_list(cfm_point, server, start, end)
  sash_list = query_to_list(sash_point, server, start, end)
  occ_list = query_to_list(occ_point, server, start, end)
  internal_temp_list = query_to_list(internal_temp_point, server, start, end)
  external_temp_list = query_to_list(external_temp_point, server, start, end)

  df = pd.concat([cfm_list, sash_list, occ_list, internal_temp_list, external_temp_list], axis=1)
  df.columns = ["cfm", "sash", "occ", "internal_temp", "external_temp"]
  display(df)

  df["external_temp"] = df["external_temp"].interpolate()
  display(df)
  
  time_interval = df.index[1].minute - df.index[0].minute

  df['BTU'] = df.apply(lambda df: coldorhot(df['cfm'], df['external_temp'], df['internal_temp'], time_interval=time_interval), axis=1)
  df.index = df.index.map(lambda x: x.to_pydatetime().replace(tzinfo=tz.tzutc()).astimezone(tz.tzlocal()))
  display(df)

  df = df[df['occ']==int(is_occupied)]

  print("\nFinal Data Frame: ")
  display(df)

  df = df.groupby(pd.Grouper(freq='60Min', label='right')).sum()

  return df["BTU"]

energy_data_occ = total_energy(cfm_point="#biotech/biotech_4th_floor/fourth_floor_fume_hood_lab_spaces/lab_433_control/hoodvalve_flow/trend_log",
             sash_point="#biotech/biotech_4th_floor/fourth_floor_fume_hood_lab_spaces/lab_433_control/hood_sash",
             occ_point="#biotech/biotech_4th_floor/fourth_floor_fume_hood_lab_spaces/lab_433_control/occ_trend",
             internal_temp_point="#biotech/biotech_4th_floor/fourth_floor_fume_hood_lab_spaces/lab_433_control/zone/zone_temp/trend_log",
             external_temp_point="#biotech/ground_flr_mech/building_hydronic_heating_syatems/reheat_heat_exchanger/oat",
             server = "biotech_main",
             start=str(datetime(2021, 11, 17)),
             end=str(datetime(2021, 11, 18)),
             is_occupied=True)

energy_data_unocc = total_energy(cfm_point="#biotech/biotech_4th_floor/fourth_floor_fume_hood_lab_spaces/lab_433_control/hoodvalve_flow/trend_log",
             sash_point="#biotech/biotech_4th_floor/fourth_floor_fume_hood_lab_spaces/lab_433_control/hood_sash",
             occ_point="#biotech/biotech_4th_floor/fourth_floor_fume_hood_lab_spaces/lab_433_control/occ_trend",
             internal_temp_point="#biotech/biotech_4th_floor/fourth_floor_fume_hood_lab_spaces/lab_433_control/zone/zone_temp/trend_log",
             external_temp_point="#biotech/ground_flr_mech/building_hydronic_heating_syatems/reheat_heat_exchanger/oat",
             server = "biotech_main",
             start=str(datetime(2021, 11, 17)),
             end=str(datetime(2021, 11, 18)),
             is_occupied=False)

print(energy_data_occ)
print(energy_data_unocc)

final_df = pd.DataFrame(data={"occ": energy_data_occ, "unocc" : energy_data_unocc})
final_df = final_df.fillna(0)
final_df

<Response [200]>

 #biotech/biotech_4th_floor/fourth_floor_fume_hood_lab_spaces/lab_433_control/hoodvalve_flow/trend_log 
 2021-11-17 05:00:00    224.62
2021-11-17 05:05:00    223.15
2021-11-17 05:10:00    224.13
2021-11-17 05:15:00    224.13
2021-11-17 05:20:00    224.62
                        ...  
2021-11-19 04:35:00    222.66
2021-11-19 04:40:00    224.13
2021-11-19 04:45:00    225.10
2021-11-19 04:50:00    223.64
2021-11-19 04:55:00    224.13
Length: 576, dtype: float64

 #biotech/biotech_4th_floor/fourth_floor_fume_hood_lab_spaces/lab_433_control/hoodvalve_flow/trend_log  new
 2021-11-17 05:00:00    224.62
2021-11-17 05:05:00    223.15
2021-11-17 05:10:00    224.13
2021-11-17 05:15:00    224.13
2021-11-17 05:20:00    224.62
                        ...  
2021-11-19 04:35:00    222.66
2021-11-19 04:40:00    224.13
2021-11-19 04:45:00    225.10
2021-11-19 04:50:00    223.64
2021-11-19 04:55:00    224.13
Length: 576, dtype: float64
<Response [200]>

 #biotech/biotech_4th_floor/fourt

,cfm,sash,occ,internal_temp,external_temp
2021-11-17 05:00:00,224.62,3.3,1.0,72.2,35.2
2021-11-17 05:05:00,223.15,3.3,1.0,72.2,NaN
2021-11-17 05:10:00,224.13,3.3,1.0,72.2,NaN
2021-11-17 05:15:00,224.13,3.3,1.0,72.2,35.3
2021-11-17 05:20:00,224.62,3.3,1.0,72.2,NaN
...,...,...,...,...,...
2021-11-19 04:35:00,222.66,2.4,0.0,70.3,NaN
2021-11-19 04:40:00,224.13,2.4,0.0,70.3,NaN
2021-11-19 04:45:00,225.10,2.4,0.0,70.3,42.7
2021-11-19 04:50:00,223.64,2.4,0.0,70.3,NaN


,cfm,sash,occ,internal_temp,external_temp
2021-11-17 05:00:00,224.62,3.3,1.0,72.2,35.200000
2021-11-17 05:05:00,223.15,3.3,1.0,72.2,35.233333
2021-11-17 05:10:00,224.13,3.3,1.0,72.2,35.266667
2021-11-17 05:15:00,224.13,3.3,1.0,72.2,35.300000
2021-11-17 05:20:00,224.62,3.3,1.0,72.2,35.366667
...,...,...,...,...,...
2021-11-19 04:35:00,222.66,2.4,0.0,70.3,42.900000
2021-11-19 04:40:00,224.13,2.4,0.0,70.3,42.800000
2021-11-19 04:45:00,225.10,2.4,0.0,70.3,42.700000
2021-11-19 04:50:00,223.64,2.4,0.0,70.3,42.700000


,cfm,sash,occ,internal_temp,external_temp,BTU
2021-11-17 00:00:00-05:00,224.62,3.3,1.0,72.2,35.200000,747.98460
2021-11-17 00:05:00-05:00,223.15,3.3,1.0,72.2,35.233333,742.42005
2021-11-17 00:10:00-05:00,224.13,3.3,1.0,72.2,35.266667,745.00812
2021-11-17 00:15:00-05:00,224.13,3.3,1.0,72.2,35.300000,744.33573
2021-11-17 00:20:00-05:00,224.62,3.3,1.0,72.2,35.366667,744.61530
...,...,...,...,...,...,...
2021-11-18 23:35:00-05:00,222.66,2.4,0.0,70.3,42.900000,549.07956
2021-11-18 23:40:00-05:00,224.13,2.4,0.0,70.3,42.800000,554.72175
2021-11-18 23:45:00-05:00,225.10,2.4,0.0,70.3,42.700000,559.14840
2021-11-18 23:50:00-05:00,223.64,2.4,0.0,70.3,42.700000,555.52176



Final Data Frame: 


...

,cfm,sash,occ,internal_temp,external_temp,BTU
2021-11-17 00:00:00-05:00,224.62,3.3,1.0,72.2,35.200000,747.98460
2021-11-17 00:05:00-05:00,223.15,3.3,1.0,72.2,35.233333,742.42005
2021-11-17 00:10:00-05:00,224.13,3.3,1.0,72.2,35.266667,745.00812
2021-11-17 00:15:00-05:00,224.13,3.3,1.0,72.2,35.300000,744.33573
2021-11-17 00:20:00-05:00,224.62,3.3,1.0,72.2,35.366667,744.61530
...,...,...,...,...,...,...
2021-11-18 23:35:00-05:00,222.66,2.4,0.0,70.3,42.900000,549.07956
2021-11-18 23:40:00-05:00,224.13,2.4,0.0,70.3,42.800000,554.72175
2021-11-18 23:45:00-05:00,225.10,2.4,0.0,70.3,42.700000,559.14840
2021-11-18 23:50:00-05:00,223.64,2.4,0.0,70.3,42.700000,555.52176



Final Data Frame: 


,cfm,sash,occ,internal_temp,external_temp,BTU
2021-11-17 01:50:00-05:00,223.15,3.3,0.0,72.2,35.733333,732.37830
2021-11-17 01:55:00-05:00,223.15,3.3,0.0,72.2,35.766667,731.70885
2021-11-17 02:00:00-05:00,223.64,3.3,0.0,72.2,35.800000,732.64464
2021-11-17 02:05:00-05:00,223.64,3.3,0.0,72.0,35.700000,730.63188
2021-11-17 02:10:00-05:00,223.64,3.3,0.0,71.9,35.600000,730.63188
...,...,...,...,...,...,...
2021-11-18 23:35:00-05:00,222.66,2.4,0.0,70.3,42.900000,549.07956
2021-11-18 23:40:00-05:00,224.13,2.4,0.0,70.3,42.800000,554.72175
2021-11-18 23:45:00-05:00,225.10,2.4,0.0,70.3,42.700000,559.14840
2021-11-18 23:50:00-05:00,223.64,2.4,0.0,70.3,42.700000,555.52176


2021-11-17 01:00:00-05:00    8897.68611
2021-11-17 02:00:00-05:00    7430.68746
2021-11-17 03:00:00-05:00    1503.62478
2021-11-17 04:00:00-05:00    8783.13078
2021-11-17 05:00:00-05:00    4472.82651
2021-11-17 06:00:00-05:00    2815.84782
2021-11-17 07:00:00-05:00    6090.31431
2021-11-17 08:00:00-05:00    2652.12810
2021-11-17 09:00:00-05:00    6343.16682
2021-11-17 10:00:00-05:00    7317.14085
2021-11-17 11:00:00-05:00    7134.71613
2021-11-17 12:00:00-05:00    6813.28503
2021-11-17 13:00:00-05:00    6341.22387
2021-11-17 14:00:00-05:00    6029.75880
2021-11-17 15:00:00-05:00    5692.98126
2021-11-17 16:00:00-05:00    7604.98143
2021-11-17 17:00:00-05:00    7453.21290
2021-11-17 18:00:00-05:00    7449.60969
2021-11-17 19:00:00-05:00    7380.94521
2021-11-17 20:00:00-05:00    7247.46801
2021-11-17 21:00:00-05:00    5845.40184
2021-11-17 22:00:00-05:00     428.57496
2021-11-17 23:00:00-05:00    2949.27858
2021-11-18 00:00:00-05:00    2537.84106
2021-11-18 01:00:00-05:00       0.00000


,occ,unocc
2021-11-17 01:00:00-05:00,8897.68611,0.00000
2021-11-17 02:00:00-05:00,7430.68746,1464.08715
2021-11-17 03:00:00-05:00,1503.62478,7331.68383
2021-11-17 04:00:00-05:00,8783.13078,0.00000
2021-11-17 05:00:00-05:00,4472.82651,4514.37108
2021-11-17 06:00:00-05:00,2815.84782,5728.68471
2021-11-17 07:00:00-05:00,6090.31431,2019.70143
2021-11-17 08:00:00-05:00,2652.12810,5349.48102
2021-11-17 09:00:00-05:00,6343.16682,1306.25478
2021-11-17 10:00:00-05:00,7317.14085,0.00000
